In [ ]:
# librerías
import os
import json
import hashlib
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce # lo usamos para los merges múltiples

In [9]:
# verificamos la ruta de los datos y si se encuentran ahi
data_path = '../data'
excel_files = glob.glob(os.path.join(data_path, '*.xlsx'))
print(f'Archivos encontrados: {len(excel_files)}')
print(excel_files)

Archivos encontrados: 3
['../data\\Totalizadores Planta de Cerveza - 2022_2023.xlsx', '../data\\Totalizadores Planta de Cerveza 2021_2022.xlsx', '../data\\Totalizadores Planta de Cerveza 2023_2024.xlsx']


In [10]:
# carga y lectura de los datos

# primero definimos las hojas que vamos a cargar
sheets_to_load = [
    'Consolidado EE',
    'Consolidado Produccion',
    'Consolidado GasVapor',
    'Totalizadores Energia'
]

print(f'Hojas a cargar: {sheets_to_load}')

# ahora vamos a leer y almacenar los datos en un diccionario
all_sheets_data = {sheet_name: [] for sheet_name in sheets_to_load}

# realizamos un pequño control para ver si los archivos existen
if 'excel_files' not in locals() or not excel_files:
    raise FileNotFoundError("No se encontraron archivos Excel en la ruta especificada.")
for file in excel_files:
    print(f'Cargando archivo: {file}')
    try:
        xls = pd.ExcelFile(file)
        for sheet_name in sheets_to_load:
            if sheet_name in xls.sheet_names:
                try:
                    df_sheet = xls.parse(sheet_name=sheet_name) # leemos la hoja
                    df_sheet.columns = df_sheet.columns.str.strip() # limpiamos los nombres de las columnas
                    all_sheets_data[sheet_name].append(df_sheet) # almacenamos el dataframe
                    print(f' Hoja {sheet_name} leída correctamente.')
                except Exception as e:
                    print(f"Error al procesar la hoja {sheet_name} en el archivo {file}: {e}")
            else:
                print(f" Hoja {sheet_name} no encontrada en el archivo {file}.")
    except Exception as e_file:
        print(f"Error al abrir el archivo {file}: {e_file}")

Hojas a cargar: ['Consolidado EE', 'Consolidado Produccion', 'Consolidado GasVapor', 'Totalizadores Energia']
Cargando archivo: ../data\Totalizadores Planta de Cerveza - 2022_2023.xlsx
 Hoja Consolidado EE leída correctamente.
 Hoja Consolidado Produccion leída correctamente.
 Hoja Consolidado GasVapor leída correctamente.
 Hoja Totalizadores Energia leída correctamente.
Cargando archivo: ../data\Totalizadores Planta de Cerveza 2021_2022.xlsx
 Hoja Consolidado EE leída correctamente.
 Hoja Consolidado Produccion leída correctamente.
 Hoja Consolidado GasVapor leída correctamente.
 Hoja Totalizadores Energia leída correctamente.
Cargando archivo: ../data\Totalizadores Planta de Cerveza 2023_2024.xlsx
 Hoja Consolidado EE leída correctamente.
 Hoja Consolidado Produccion leída correctamente.
 Hoja Consolidado GasVapor leída correctamente.
 Hoja Totalizadores Energia leída correctamente.


In [19]:
# combinamos los dataframes de cada hoja
concatenated_data = {}
for sheet_name, df_list in all_sheets_data.items():
    if df_list:  # verificamos que la lista no esté vacía
        concatenated_data[sheet_name] = pd.concat(df_list, ignore_index=True)
        print(f'Hoja {sheet_name} concatenada: {concatenated_data[sheet_name].shape[0]} filas, {concatenated_data[sheet_name].shape[1]} columnas.')
    else:
        print(f'No se encontraron datos para la hoja {sheet_name}.')

# verificamos que los dataframes concatenados existan y usamos 'Consolidado EE' como base para el merge
base_sheet = sheets_to_load[0]
if base_sheet in concatenated_data:
    # preparamos antes los nombres de las columnas para dataframe base
    concatenated_data[base_sheet].columns = concatenated_data[base_sheet].columns.str.strip() # limpiamos los nombres de las columnas
    df_unified_raw = concatenated_data[base_sheet].copy() # empezamos con la hoja del target
    print(f'Dataframe base {base_sheet} preparado para merge: {df_unified_raw.shape[0]} filas, {df_unified_raw.shape[1]} columnas.')
    
    # ahora iteramos sobre las otras hojas para hacer los merges
    for sheet_name in sheets_to_load[1:]: # saltamos la primera hoja que ya usamos
        if sheet_name in concatenated_data:
            try:
                # preparamos los nombres de las columnas para el dataframe a mergear
                concatenated_data[sheet_name].columns = concatenated_data[sheet_name].columns.str.strip() # limpiamos los nombres de las columnas
                df_to_merge = concatenated_data[sheet_name]

                # para que estén ordenados por fecha, nos aseguramos que 'DIA' y 'HORA' existan y sean del tipo datetime
                if 'DIA' in df_to_merge.columns and 'HORA' in df_to_merge.columns:
                    df_to_merge['DIA'] = pd.to_datetime(df_to_merge['DIA'], errors='coerce')
                    # no convertimos 'HORA' y nos aseguramos que 'DIA' en el dataframe base también sea datetime
                    if 'DIA' in df_unified_raw.columns:
                        df_unified_raw['DIA'] = pd.to_datetime(df_unified_raw['DIA'], errors='coerce')
                    else:
                        print(f"La columna 'DIA' no existe en el dataframe base.")
                    
                    # seleccionamos la columnas unicas de estas hojas, excluyendo 'DIA' y 'HORA'
                    unique_columns = [col for col in df_to_merge.columns if col not in ['DIA', 'HORA']]
                    merge_columns = ['DIA', 'HORA'] + unique_columns
                    
                    # realizamos el merge
                    df_unified_raw = pd.merge(
                        df_unified_raw,
                        df_to_merge[merge_columns].drop_duplicates(subset=['DIA', 'HORA']),# evitamos duplicados en las claves de merge
                        on=['DIA', 'HORA'],
                        how='left'  # usamos left join para mantener todas las filas del dataframe base
                    )
                    print(f'Merge con hoja {sheet_name} realizado: {df_unified_raw.shape[0]} filas, {df_unified_raw.shape[1]} columnas.')
                else:
                    print(f"Las columnas 'DIA' y/o 'HORA' no existen en la hoja {sheet_name}.")
            except Exception as e_merge:
                print(f"Error al hacer merge con la hoja {sheet_name}: {e_merge}")
        else:
            print(f'No se encontraron datos para la hoja {sheet_name}, no se realizó merge.')
    
    """ mostramos el resultado final del dataframe unificado y lo guardamos
    if 'df_unified_raw' in locals() and not df_unified_raw.empty:
        print(f'Resultado final del dataframe unificado: {df_unified_raw.shape[0]} filas, {df_unified_raw.shape[1]} columnas.')
        print('\nPrimeras filas del dataframe unificado:')
        print(df_unified_raw.head())
        # guardamos el dataframe unificado en un archivo CSV
        output_file = os.path.join(data_path, 'df_unified_raw.csv')
        try:
            df_unified_raw.to_csv(output_file, index=False, decimal='.')
            print(f'Dataframe unificado guardado en {output_file}')
        except Exception as e_save:
            print(f"Error al guardar el dataframe unificado: {e_save}")
    else:
        print('\n No se pudo crear el dataframe unificado.')
        df_unified_raw = pd.DataFrame()  # creamos un dataframe vacío para evitar errores posteriores
    """
else:
    print(f'No se encontró la hoja base {base_sheet} en los datos concatenados.')
    df_unified_raw = pd.DataFrame()  # creamos un dataframe vacío para evitar errores posteriores

Hoja Consolidado EE concatenada: 43515 filas, 24 columnas.
Hoja Consolidado Produccion concatenada: 43034 filas, 19 columnas.
Hoja Consolidado GasVapor concatenada: 43520 filas, 21 columnas.
Hoja Totalizadores Energia concatenada: 42900 filas, 60 columnas.
Dataframe base Consolidado EE preparado para merge: 43515 filas, 24 columnas.
Merge con hoja Consolidado Produccion realizado: 43515 filas, 41 columnas.
Merge con hoja Consolidado GasVapor realizado: 43515 filas, 60 columnas.
Merge con hoja Totalizadores Energia realizado: 43515 filas, 118 columnas.


In [20]:
print('\nInformación del DataFrame unificado TOTAL:')
pd.options.display.max_info_columns = df_unified_raw.shape[1] + 1 # Mostrar todas las columnas
df_unified_raw.info()


Información del DataFrame unificado TOTAL:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43515 entries, 0 to 43514
Data columns (total 118 columns):
 #    Column                    Non-Null Count  Dtype         
---   ------                    --------------  -----         
 0    DIA                       42900 non-null  datetime64[ns]
 1    HORA                      42900 non-null  object        
 2    Planta (Kw)               42902 non-null  float64       
 3    Elaboracion (Kw)          42902 non-null  float64       
 4    Bodega (Kw)               42902 non-null  float64       
 5    Cocina (Kw)               42902 non-null  float64       
 6    Envasado (Kw)             42902 non-null  float64       
 7    Linea 2 (Kw)              42894 non-null  float64       
 8    Linea 3 (Kw)              42902 non-null  float64       
 9    Linea 4 (Kw)              42902 non-null  float64       
 10   Servicios (Kw)            42902 non-null  float64       
 11   Sala Maq (Kw)        

In [21]:
# procedemos a realizar una limpieza básica del dataframe unificado

# 1. identificamos y eliminamos columnas redundantes
columns_unnamed = [col for col in df_unified_raw.columns if 'Unnamed' in col]
cols_id = [col for col in df_unified_raw.columns if col.lower() == 'id']
# de 'Consolidado EE' sabemos que 'Fecha/Hora_x' y 'Kw de Frio' no sirven
cols_to_drop_ee = ['Fecha/Hora_x', 'Kw de Frio']
cols_to_drop = columns_unnamed + cols_id + cols_to_drop_ee # columnas a eliminar

# ahora filtramos solo las que realmente existen en el dataframe
existing_cols_to_drop = [col for col in cols_to_drop if col in df_unified_raw.columns]
print(f'\nColumnas a eliminar: {existing_cols_to_drop}')
df_unified_cleaned = df_unified_raw.drop(columns=existing_cols_to_drop)
print(f'Dataframe después de eliminar columnas redundantes: {df_unified_cleaned.shape[0]} filas, {df_unified_cleaned.shape[1]} columnas.')


Columnas a eliminar: ['Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Id', 'Fecha/Hora_x', 'Kw de Frio']
Dataframe después de eliminar columnas redundantes: 43515 filas, 104 columnas.


In [22]:
# 2. corregimos las columnas con sufijos _x y _y generados por los merges

cols_with_suffixes = [col for col in df_unified_cleaned.columns if col.endswith('_x') or col.endswith('_y')] # identificamos columnas con sufijos
base_with_suffixes = sorted(list(set([col.replace('_x', '').replace('_y', '') for col in cols_with_suffixes]))) # identificamos las bases de esas columnas

print(f'\nColumnas con sufijos a corregir: {cols_with_suffixes}')
print(f'Bases de columnas con sufijos: {base_with_suffixes}')

cols_with_suffixes_to_drop = [] # columnas a eliminar después de la corrección
for base in base_with_suffixes:
    col_x = base + '_x'
    col_y = base + '_y'
    cols_to_compare = []
    if col_x in df_unified_cleaned.columns:
        cols_to_compare.append(col_x)
    if col_y in df_unified_cleaned.columns:
        cols_to_compare.append(col_y)
    
    if len(cols_to_compare) == 2:
        # comparamos si son (casi) iguales , ignorando NaNs
        are_equal = df_unified_cleaned[col_x].equals(df_unified_cleaned[col_y])
        # aca nos vamos a quedar con _x (de Consolidado EE) y eliminamos _y
        cols_to_keep = col_x
        cols_to_rename = base
        cols_to_drop = col_y
        df_unified_cleaned.rename(columns={cols_to_keep: cols_to_rename}, inplace=True)
        cols_with_suffixes_to_drop.append(cols_to_drop)
    elif len(cols_to_compare) == 1:
        # si solo existe una de las dos, la renombramos
        existing_col = cols_to_compare[0]
        cols_to_rename = base
        df_unified_cleaned.rename(columns={existing_col: cols_to_rename}, inplace=True)

# eliminamos todas las columnas _y que quedaron
if cols_with_suffixes_to_drop:
    df_unified_cleaned.drop(columns=cols_with_suffixes_to_drop, inplace=True)


Columnas con sufijos a corregir: ['KW Gral Planta_x', 'KW CO2_x', 'Fecha/Hora_y', 'KW Gral Planta_y', 'KW CO2_y']
Bases de columnas con sufijos: ['Fecha/Hora', 'KW CO2', 'KW Gral Planta']


In [23]:
# 3. limpiamos las columnas de distinto tipo (object a numérico)
cols_to_clean = 'KW Trafo 8'
if cols_to_clean in df_unified_cleaned.columns:
    original_dtype = df_unified_cleaned[cols_to_clean].dtype
    nulls_before = df_unified_cleaned[cols_to_clean].isnull().sum()
    # intentamos convertir a numérico, forzando errores a NaN
    df_unified_cleaned[cols_to_clean] = pd.to_numeric(df_unified_cleaned[cols_to_clean], errors='coerce')
    nulls_after = df_unified_cleaned[cols_to_clean].isnull().sum()
    new_dtype = df_unified_cleaned[cols_to_clean].dtype
    if nulls_after > nulls_before:
        print(f' se produjeron {nulls_after - nulls_before} nuevos NaNs al convertir la columna {cols_to_clean} de {original_dtype} a {new_dtype}.')
else:
    print(f'\n La columna {cols_to_clean} no existe en el dataframe para limpiar.') 

 se produjeron 8 nuevos NaNs al convertir la columna KW Trafo 8 de object a float64.


In [24]:
# ahora mostramos como quedo quedó el dataframe "limpio"
print(f'Dimensiones del DataFrame limpio: {df_unified_cleaned.shape[0]} filas, {df_unified_cleaned.shape[1]} columnas.')
print('\nPrimeras filas del DataFrame limpio:')
print(df_unified_cleaned.head())
print('\nInformación del DataFrame limpio:')
df_unified_cleaned.info()

Dimensiones del DataFrame limpio: 43515 filas, 102 columnas.

Primeras filas del DataFrame limpio:
         DIA      HORA  Planta (Kw)  Elaboracion (Kw)  Bodega (Kw)  \
0 2022-07-01  02:00:00      1368.76              46.0        101.5   
1 2022-07-01  03:00:00      2765.64              93.0        203.5   
2 2022-07-01  04:00:00      4124.46             141.0        306.0   
3 2022-07-01  05:00:01      5419.31             182.5        401.5   
4 2022-07-01  06:00:01      6673.19             228.0        499.0   

   Cocina (Kw)  Envasado (Kw)  Linea 2 (Kw)  Linea 3 (Kw)  Linea 4 (Kw)  ...  \
0          7.0           12.0         30.01          42.0           0.0  ...   
1         13.0           26.0         59.64          84.0           0.0  ...   
2         20.0           38.0         89.21         124.0           0.0  ...   
3         26.0           50.0        117.81         164.0           0.0  ...   
4         33.0           62.0        146.19         202.0           0.0  ...   


In [25]:
# Reasignamos el dataframe para que apunte a la versión limpia
df_unified_raw = df_unified_cleaned

if 'df_unified_raw' in locals() and not df_unified_raw.empty:
        print(f'Resultado final del dataframe unificado: {df_unified_raw.shape[0]} filas, {df_unified_raw.shape[1]} columnas.')
        print('\nPrimeras filas del dataframe unificado:')
        print(df_unified_raw.head())
        # guardamos el dataframe unificado en un archivo CSV
        output_file = os.path.join(data_path, 'df_unified_raw.csv')
        try:
            df_unified_raw.to_csv(output_file, index=False, decimal='.')
            print(f'Dataframe unificado guardado en {output_file}')
        except Exception as e_save:
            print(f"Error al guardar el dataframe unificado: {e_save}")
else:
        print('\n No se pudo crear el dataframe unificado.')
        df_unified_raw = pd.DataFrame()  # creamos un dataframe vacío para evitar errores posteriores

Resultado final del dataframe unificado: 43515 filas, 102 columnas.

Primeras filas del dataframe unificado:
         DIA      HORA  Planta (Kw)  Elaboracion (Kw)  Bodega (Kw)  \
0 2022-07-01  02:00:00      1368.76              46.0        101.5   
1 2022-07-01  03:00:00      2765.64              93.0        203.5   
2 2022-07-01  04:00:00      4124.46             141.0        306.0   
3 2022-07-01  05:00:01      5419.31             182.5        401.5   
4 2022-07-01  06:00:01      6673.19             228.0        499.0   

   Cocina (Kw)  Envasado (Kw)  Linea 2 (Kw)  Linea 3 (Kw)  Linea 4 (Kw)  ...  \
0          7.0           12.0         30.01          42.0           0.0  ...   
1         13.0           26.0         59.64          84.0           0.0  ...   
2         20.0           38.0         89.21         124.0           0.0  ...   
3         26.0           50.0        117.81         164.0           0.0  ...   
4         33.0           62.0        146.19         202.0           0.

In [26]:
# procedemos a calcular el checksum del dataframe unificado

# nos aseguramos que df_unified_raw exista y no esté vacío
if 'df_unified_raw' in locals() and not df_unified_raw.empty:
    # ordenamos las columnas para asegurar consistencia
    if 'DIA' in df_unified_raw.columns and 'HORA' in df_unified_raw.columns:
        df_unified_raw['HORA'] = df_unified_raw['HORA'].astype(str)
        df_unified_raw = df_unified_raw.sort_values(by=['DIA', 'HORA']).reset_index(drop=True) # ordenamos por fecha y hora
    elif 'DIA' in df_unified_raw.columns:
        df_unified_raw = df_unified_raw.sort_values(by=['DIA']).reset_index(drop=True) # ordenamos solo por fecha si 'HORA' no existe
    else:
        print("La columna 'DIA' no existe en el dataframe, no se puede ordenar.")
    
    # convertimos a bytes para hashear
    data_bytes = df_unified_raw.to_csv(index=False, decimal='.').encode()
    checksum = hashlib.md5(data_bytes).hexdigest()
    
    # lo guardamos
    checksum_file_path = os.path.join(data_path, 'checksum.json')
    checksum_data = {}
    try:
        with open(checksum_file_path, 'r') as f:
            checksum_data = json.load(f)
    except FileNotFoundError:
        print(f'Archivo de checksum no encontrado, se creará uno nuevo en {checksum_file_path}.')
    except json.JSONDecodeError:
        print(f'Error al decodificar el archivo de checksum, se creará uno nuevo en {checksum_file_path}.')
    
    checksum_data['df_unified_raw'] = checksum # actualizamos o agregamos el checksum
    
    # guardamos el archivo
    try:
        with open(checksum_file_path, 'w') as f:
            json.dump(checksum_data, f, indent=4)
        print(f'Checksum guardado en {checksum_file_path}: {checksum}')
    except Exception as e_checksum:
        print(f"Error al guardar el checksum: {e_checksum}")
else:
    print('\n No se pudo calcular el checksum del dataframe unificado porque no existe o está vacío.')

Archivo de checksum no encontrado, se creará uno nuevo en ../data\checksum.json.
Checksum guardado en ../data\checksum.json: 112ef4a3088ea4942f0c57797aa99d9e


In [27]:
# unificación del dataframe con el ultimo registro de cada día

if 'df_unified_raw' in locals() and not df_unified_raw.empty:
    # primero verificamos nuevamente que 'DIA' sea del tipo datetime (a modo de seguro)
    if not pd.api.types.is_datetime64_any_dtype(df_unified_raw['DIA']):
        df_unified_raw['DIA'] = pd.to_datetime(df_unified_raw['DIA'], errors='coerce')
        
    # creamos columnas temporales para la agregación
    df_unified_raw['Fecha_DT_Temp'] = df_unified_raw['DIA'].dt.date
    
    # convertimos 'HORA' a timedelta para facilitar la comparación
    df_unified_raw['HORA_TD_Temp'] = pd.to_timedelta(df_unified_raw['HORA'].astype(str), errors='coerce')
    
    # combinamos 'DIA' y 'HORA' en una sola columna datetime para identificar el último registro del día
    df_unified_raw['Timestamp_Completo_Temp'] = pd.to_datetime(df_unified_raw['Fecha_DT_Temp']) + df_unified_raw['HORA_TD_Temp']
    
    # lógica para obtener el último registro de cada día
    
    # 1. filtramos las filas donde Timestamp_Completo_Temp es inválido O Fecha_DT_Temp es NaT
    df_valid = df_unified_raw.dropna(subset=['Fecha_DT_Temp', 'Timestamp_Completo_Temp']).copy()
    if df_valid.shape[0] < df_unified_raw.shape[0]:
        print(f"INFO: Se procesarán {df_valid.shape[0]} filas con timestamps válidos (excluyendo {df_unified_raw.shape[0] - df_valid.shape[0]} inválidas).")
    
    # 2. obtenemos el índice del último registro por día
    idx_last_hours = df_valid.groupby('Fecha_DT_Temp')['Timestamp_Completo_Temp'].idxmax()
    print(f"INFO: Se encontraron {len(idx_last_hours)} días únicos con registros válidos.")
    
    # 3. usamos los índices para seleccionar las filas correspondientes
    df_daily = df_valid.loc[idx_last_hours].reset_index(drop=True)
    
    # creamos la columna unicamente con la fecha (sin hora)
    df_daily['Fecha'] = pd.to_datetime(df_daily['Fecha_DT_Temp'])
    df_daily.set_index('Fecha', inplace=True)
    
    # eliminamos las columnas temporales
    df_daily.drop(columns=['DIA','HORA','Fecha_DT_Temp', 'HORA_TD_Temp', 'Timestamp_Completo_Temp'], inplace=True)
    
    # guardamos el dataframe diario
    output_daily_file = os.path.join(data_path, 'df_daily.csv')
    try:
        df_daily.to_csv(output_daily_file, index=True, decimal='.')
        print(f'Dataframe diario guardado en {output_daily_file}: {df_daily.shape[0]} filas, {df_daily.shape[1]} columnas.')
    except Exception as e_save_daily:
        print(f"Error al guardar el dataframe diario: {e_save_daily}")
else:
    print('\n No se pudo crear el dataframe diario porque el dataframe unificado no existe o está vacío.')

INFO: Se procesarán 42900 filas con timestamps válidos (excluyendo 615 inválidas).
INFO: Se encontraron 1199 días únicos con registros válidos.
Dataframe diario guardado en ../data\df_daily.csv: 1199 filas, 100 columnas.


In [28]:
# hacemos un nuevo checksum para el dataframe diario
if 'df_daily' in locals() and not df_daily.empty:
    # ordenamos por índice (Fecha) para consistencia
    df_daily = df_daily.sort_index().reset_index()
    
    # convertimos a bytes para hashear
    data_daily_bytes = df_daily.to_csv(index=False, decimal='.').encode()
    checksum_daily = hashlib.md5(data_daily_bytes).hexdigest()
    
    # lo guardamos
    checksum_file_path = os.path.join(data_path, 'checksum.json')
    checksum_data = {}
    try:
        with open(checksum_file_path, 'r') as f:
            checksum_data = json.load(f)
    except FileNotFoundError:
        print(f'Archivo de checksum no encontrado, se creará uno nuevo en {checksum_file_path}.')
    except json.JSONDecodeError:
        print(f'Error al decodificar el archivo de checksum, se creará uno nuevo en {checksum_file_path}.')
    
    checksum_data['df_daily'] = checksum_daily # actualizamos o agregamos el checksum
    
    # guardamos el archivo
    try:
        with open(checksum_file_path, 'w') as f:
            json.dump(checksum_data, f, indent=4)
        print(f'Checksum del dataframe diario guardado en {checksum_file_path}: {checksum_daily}')
    except Exception as e_checksum_daily:
        print(f"Error al guardar el checksum del dataframe diario: {e_checksum_daily}")

Checksum del dataframe diario guardado en ../data\checksum.json: 59e86be030e880a8b735e2f2c5a4e6fd
